# Introduction to TensorFlow
In this notebook we will go through the basics in TensorFlow, build a simple multilayer perceptron and try out the TensorBoard visualization utility.

## Basics
Here we will see how to write a simple Hello, World! application in TensorFlow and  learn about the basics.


In [ ]:
import tensorflow as tf
import numpy as np

### Tensors


The basic building blocks of TensorFlow are **Tensors**. Tensors can hold an array of data, very similar to Numpy's `ndarray`. It also has many of the same attributes as an ndarray, like `dtype`, `shape`, `ndim`.


Let's see some examples:

In [ ]:
a = tf.constant(3.14)
b = tf.constant(42, dtype=tf.float32)
c = tf.constant([[1, 2]])

print("a =", a)
print("b =", b)
print("c =", c)
print("Shape of c:", c.shape)
print()

# You can also create Tensors from ndarrays:
print(tf.constant(np.ones((2,2), dtype='float32')))
print()

# But tf.ones is also available
print(tf.ones((2,2), dtype='float32'))

a = tf.Tensor(3.14, shape=(), dtype=float32)
b = tf.Tensor(42.0, shape=(), dtype=float32)
c = tf.Tensor([[1 2]], shape=(1, 2), dtype=int32)
Shape of c: (1, 2)

tf.Tensor(
[[1. 1.]
 [1. 1.]], shape=(2, 2), dtype=float32)

tf.Tensor(
[[1. 1.]
 [1. 1.]], shape=(2, 2), dtype=float32)


Tensors supports many builtin functions, broadcasting and also indexing - pretty much equivalent to Numpy.

In [ ]:
print(a + b)
print(tf.add(a, b))  # This is the same as the previous

print(a * b)

# Broadcasting
x = tf.constant(3)
mul = tf.multiply(x, c)
print(mul)

# Indexing:
print(c[0, 1])

tf.Tensor(45.14, shape=(), dtype=float32)
tf.Tensor(45.14, shape=(), dtype=float32)
tf.Tensor(131.88, shape=(), dtype=float32)
tf.Tensor([[3 6]], shape=(1, 2), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)


However, Numpy can execute only on CPU, while Tensor operations are usually executed on the GPU. The `device` attribute tells us where a tensor is stured (CPU or GPU memory):

In [ ]:
a = tf.constant(3)
b = tf.constant(3)
result = tf.add(a, b)

print("a.device:", a.device)
print("b.device:", b.device)
print("result.device:", result.device)  # The result is on the GPU, as operations are performed on the GPU

a.device: /job:localhost/replica:0/task:0/device:CPU:0
b.device: /job:localhost/replica:0/task:0/device:CPU:0
result.device: /job:localhost/replica:0/task:0/device:GPU:0


### Variables

Tensors are immutable objects. Tensorflow provides `Variable` whose value can be changed.

In [ ]:
v = tf.Variable(1.)
print(v)

v.assign(2.)
print(v)

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=1.0>
<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.0>


### Calculating gradients

The magic of Tensorflow happens in `GradientTape` - it records performed operations (on a "tape"), then the `gradient` function calculates the gradient ("plays the tape backwards").

For example, the following snippets calculates the derivative of `x^2` at 3:

In [ ]:
x = tf.Variable(3.)
second_order = False
with tf.GradientTape() as g:
  f = x*x


print(g.gradient(f, x))  # differentiate f with regards to x

tf.Tensor(6.0, shape=(), dtype=float32)


Note: by default GradientTape records only operations that depends on a `Variable`. If you want to include some other tensors as well, use `GradientTape.watch()`

## Building  a Multilayer Perceptron

Here will see how to build a simple neural network with TensorFlow using only the low-level API.
![](http://cs231n.github.io/assets/nn1/neural_net2.jpeg)  

It will classify handwritten digits from MNIST. The network will have two dense layers and a final softmax layer with 10 outputs.

In [ ]:
# First load the MNIST dataset
from tensorflow.keras.datasets import mnist

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize the training and test data
x_train = x_train / 255.0
x_test = x_test / 255.0

# Flatten input images into a vector
x_train = x_train.reshape(-1, 28*28).astype('float32')
x_test = x_test.reshape(-1, 28*28).astype('float32')

# use one-hot encoding for y:
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

In [ ]:
# Parameters
learning_rate = 0.01
num_steps = 500
batch_size = 32
display_step = 50  # print loss every 50 iterations

# Network Parameters
n_hidden_1 = 256 # 1st layer number of neurons
n_hidden_2 = 256 # 2nd layer number of neurons
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

In [ ]:
# Store layer weights & biases
params = {
  'w1': tf.Variable(tf.random.normal([n_input, n_hidden_1]), name='W1'),
  'w2': tf.Variable(tf.random.normal([n_hidden_1, n_hidden_2]), name='W2'),
  'w3': tf.Variable(tf.random.normal([n_hidden_2, n_classes]), name='W3'),
  'b1': tf.Variable(tf.random.normal([n_hidden_1]), name='b1'),
  'b2': tf.Variable(tf.random.normal([n_hidden_2]), name='b2'),
  'b3': tf.Variable(tf.random.normal([n_classes]), name='b3')
}

In [ ]:
# Create the model
def neural_net(x):
  # A fully connected layer with 256 neurons.
  # Remember, a fully connected layer's formula is
  #     o = g(x*W+b)
  # where g is the activation function, W is weight matrix and b is the bias vector.
  
  # Equivalent to:
  # layer_1_out = Dense(n_hidden_1, activation='relu')(x)
  a1 = tf.add(tf.matmul(x, params['w1']), params['b1'])  # shape: (batch_size, n_hidden_1)
  layer_1_out = tf.nn.relu(a1)  
  
  a2 = tf.multiply(tf.matmul(layer_1_out, params['w2']), params['b2'])  # shape: (batch_size, n_hidden_2)
  layer_2_out = tf.nn.relu(a2)
  
  layer_3_out = tf.matmul(layer_2_out, params['w3']) + params['b3']  # shape: (batch_size, n_classes)
  return tf.nn.softmax(layer_3_out)

In [ ]:
loss_fn = tf.keras.losses.CategoricalCrossentropy()

for step in range(num_steps):
  # STEPth 32 vectors from x_train/y_train
  batch_x = x_train[step*batch_size:(step+1)*batch_size]
  batch_y = y_train[step*batch_size:(step+1)*batch_size]

  with tf.GradientTape() as tape:
    pred = neural_net(batch_x)
    loss = loss_fn(batch_y, pred)

    # Calculate gradients  
    grads = tape.gradient(loss, params)

  # Update the weights
  for name in params:
     #  params[name] -= learning_rate * grads[name]
      params[name].assign_sub(learning_rate * grads[name])  # subtract lr*gradient from each weight

  if (step+1) % display_step == 0:
     # calculate performance on training set
     pred = neural_net(x_train)
     pred_class = np.argmax(pred, axis=1)
     gt_class = np.argmax(y_train, axis=1)

     acc = np.mean(gt_class==pred_class)

     print("Step " + str(step) + ", Minibatch Loss= " + \
            "{:.4f}".format(loss) + ", Training Accuracy= " + \
            "{:.3f}".format(acc))

Step 49, Minibatch Loss= 12.5923, Training Accuracy= 0.133
Step 99, Minibatch Loss= 14.1033, Training Accuracy= 0.168
Step 149, Minibatch Loss= 14.1033, Training Accuracy= 0.172
Step 199, Minibatch Loss= 13.0960, Training Accuracy= 0.172
Step 249, Minibatch Loss= 12.0886, Training Accuracy= 0.181
Step 299, Minibatch Loss= 14.6070, Training Accuracy= 0.187
Step 349, Minibatch Loss= 14.6070, Training Accuracy= 0.170
Step 399, Minibatch Loss= 12.5923, Training Accuracy= 0.181
Step 449, Minibatch Loss= 14.1033, Training Accuracy= 0.181
Step 499, Minibatch Loss= 12.5923, Training Accuracy= 0.181


## TensorBoard
With TensorBoard you can easily monitor the learning process. Unfortunately in Google Colab you cannot directly use TensorBoard, first you have to tunnel it to become  remotely accessible.

### Setup
By running the following commands you can tunnel TensorBoard to the outside world.

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2019-04-07 08:58:31--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.200.123.104, 52.203.66.95, 52.22.145.207, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.200.123.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14977695 (14M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  14.28M  14.3MB/s    in 1.0s    

2019-04-07 08:58:32 (14.3 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [14977695/14977695]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


The following will give you an url. By opening it you can access your remote TensorBoard.

Modify *LOG_DIR* if needed.

In [ ]:
LOG_DIR = './log_test'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6007 &'
    .format(LOG_DIR)
)

get_ipython().system_raw('./ngrok http 6007 &')

!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://2427574d.ngrok.io


### Visualization
You can also visualize the computational graph by using TensorBoard. First you have to save the graph to a summary file:


In [ ]:
writer = tf.summary.FileWriter('./log_test')
writer.add_graph(tf.get_default_graph())
writer.flush()

Now open the above link and select Graph in the drop-down menu in the top right corner. It will show your computational graph.

If you are running this notebook on your own machine, you can just use the following command:

In [ ]:
!tensorboard --logdir ./log_test

## High-level API
Keras is also available as a high-level API for TensorFlow. You can freely mix Keras layers and your custom layers.

Before you move on runtime reset may be necessary because of the visualization. You can do it at: **Runtime -> Reset all runtimes...**. (Warning: You will lose all data from your virtual machine.) After that do not run any previous block except the [remote setup](#scrollTo=9cdJm9TXGdkL).

#### TensorBoard Callback
Using **[tf.keras.callbacks.TensorBoard](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/TensorBoard)** we can easily log information during training, that can be later visualized using TensorBoard. It allows you to visualize dynamic graphs of your training and test metrics, as well as activation histograms for the different layers in your model.

**Main arguments:**

*  **log_dir:** the path of the directory where to save the log files to be parsed by TensorBoard.
* **histogram_freq:** frequency (in epochs) at which to compute activation and weight histograms for the layers of the model. If set to 0, histograms won't be computed. Validation data (or split) must be specified for histogram visualizations.
* **batch_size:** size of batch of inputs to feed to the network for histograms computation.
* **write_graph:** whether to visualize the graph in TensorBoard. The log file can become quite large when write_graph is set to True.
* **write_grads:** whether to visualize gradient histograms in TensorBoard.  histogram_freq must be greater than 0.
* **write_images:** whether to write model weights to visualize as image in TensorBoard.


In [ ]:
from tensorflow.keras.callbacks import TensorBoard

batch_size = 32

tbCallBack = TensorBoard(log_dir='./log_keras', 
                         histogram_freq=1,
                         batch_size=batch_size,
                         write_graph=True,
                         write_grads=True,
                         write_images=True)

#### Visualization
With TensorBoard you can see details about the network during and after training. 

In [ ]:
LOG_DIR = './log_keras'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6007 &'
    .format(LOG_DIR)
)

get_ipython().system_raw('./ngrok http 6007 &')

!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://953864d8.ngrok.io


#### Build an MLP
But for now we have nothing to see, so let's build and train a simple neural network. During training TensorBoard will automatically update.

In [ ]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5, batch_size=batch_size, 
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[tbCallBack])
model.evaluate(x_test, y_test)

11493376/11490434 [==============================] - 0s 0us/step
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 17s 277us/sample - loss: 0.2968 - acc: 0.9129 - val_loss: 0.1430 - val_acc: 0.9577
Epoch 2/5
60000/60000 [==============================] - 15s 249us/sample - loss: 0.1423 - acc: 0.9577 - val_loss: 0.1050 - val_acc: 0.9672
Epoch 3/5
60000/60000 [==============================] - 17s 282us/sample - loss: 0.1062 - acc: 0.9678 - val_loss: 0.0848 - val_acc: 0.9744
Epoch 4/5
60000/60000 [==============================] - 15s 255us/sample - loss: 0.0880 - acc: 0.9726 - val_loss: 0.0750 - val_acc: 0.9764
Epoch 5/5
10000/10000 [==============================] - 0s 44us/sample - loss: 0.0766 - acc: 0.9766


[0.07656257087504491, 0.9766]

## References
* Low-level introduction: https://www.tensorflow.org/guide/low_level_intro
* Tutorials: https://www.tensorflow.org/tutorials/
* Examples: https://github.com/aymericdamien/TensorFlow-Examples
* TensorBoard in Google Colab: https://www.dlology.com/blog/quick-guide-to-run-tensorboard-in-google-colab/
